In [ ]:
%matplotlib inline

In [ ]:
import os
import time
import xarray as xr
import numpy as np
from IPython import display

import redis

import matplotlib.pyplot as plt

In [ ]:
# Python interface to redis server
r = redis.StrictRedis(host='127.0.0.1', port=6379, db=0,
                      password=os.getenv('REDIS_PASSWORD'))

In [ ]:
# get data function
def get(key='A', shape=(22, 22)):
    key, vals = r.brpop(key)
    data = np.fromstring(vals, dtype='<i4').reshape(shape, order='F')
    return xr.DataArray(data,  dims=('x', 'y'), name=key)

In [ ]:
# figure
fig, axes = plt.subplots(ncols=3, figsize=(12, 4), sharex=True, sharey=True, )
fig.tight_layout(w_pad=3.0)
old = get()

old.plot.pcolormesh(ax=axes[0], vmin=-1, vmax=1, cmap='viridis')
old.plot.pcolormesh(ax=axes[1], vmin=-1, vmax=1, cmap='viridis')
old.plot.pcolormesh(ax=axes[2], vmin=-1, vmax=1, cmap='RdBu')

counter = 1
while True:
    new = get()
    old.plot.pcolormesh(ax=axes[0], vmin=-1, vmax=1, cmap='viridis', add_colorbar=False)
    new.plot.pcolormesh(ax=axes[1], vmin=-1, vmax=1, cmap='viridis', add_colorbar=False)
    (new - old).plot.pcolormesh(ax=axes[2], vmin=-1, vmax=1, cmap='RdBu', add_colorbar=False)
    axes[0].set_title('Old: %d' % counter)
    axes[1].set_title('New: %d' % (counter + 1))
    axes[2].set_title('Diff: %d' % (counter + 1))
    display.clear_output(wait=True)
    display.display(fig)
    time.sleep(1)
    old = new
    del new
    counter += 1

    